In [2]:
from pip._internal import main
main(["install","progressbar"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

Requirement already satisfied: progressbar in e:\python\lib\site-packages (2.5)

WARNING: You are using pip version 22.0.4; however, version 22.1.1 is available.
You should consider upgrading via the 'E:\Python\python.exe -m pip install --upgrade pip' command.


0

## 导入库

In [3]:
import progressbar
import time
import os
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import numpy as np
import librosa
import DataCollection as dc # 本地模块

## 获取标注文件

In [4]:
data = pd.read_csv("E:/Project/Data/UrbanSound8K/metadata/UrbanSound8K.csv")
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [5]:
data.shape

(8732, 8)

In [7]:
###data = data.sort_values(by=['fold', 'classID',"fsID"], ascending=[True, True, True])
dataset = np.zeros(shape = (data.shape[0],2),dtype = object)
dataset.shape

(8732, 2)

In [9]:
dataset

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=object)

## 特征提取

In [16]:
bar = progressbar.ProgressBar(maxval=data.shape[0], widgets=[progressbar.Bar('$', '||', '||'), ' ', progressbar.Percentage()])
bar.start()
for i in range(data.shape[0]):
    
    fullpath, class_id = dc.path_class(data,data.slice_file_name[i])
    try:
        X, sample_rate = librosa.load('E:/Project/Data/'+fullpath, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    except Exception:
        print("Error encountered while parsing file: ", file)
        mfccs,class_id = None, None
    feature = mfccs
    label = class_id
    dataset[i,0],dataset[i,1] = feature,label
    
    bar.update(i+1)


E:\Python\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323
  return f(*args, **kwargs)
E:\Python\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
E:\Python\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)


## 将特征以npy方式存储

In [17]:
np.save("dataset_melspectrogram",dataset,allow_pickle=True)

In [10]:
l = np.load("dataset_melspectrogram.npy",allow_pickle=True)

In [11]:
l.shape

(8732, 2)

In [13]:
l[855,0]

array([-167.33153  ,  110.32705  ,  -76.09589  ,  -30.782467 ,
        -30.730064 ,    8.0865755,    3.3659456,   25.157713 ,
         -4.7172413,    9.027242 ,   -5.0951777,   16.529928 ,
         -1.883552 ,    6.6163974,  -11.156658 ,    6.9693623,
         -7.679139 ,    2.660057 ,   -2.3932617,    2.343756 ,
        -13.88474  ,   -3.9746044,   -8.529014 ,    1.3399175,
          3.1648097,   -3.2728138,   -8.082373 ,    4.026173 ,
          3.7668002,    8.095021 ,    4.065724 ,    5.2892485,
          7.056347 ,   12.222374 ,   15.972188 ,   15.420131 ,
          4.9710736,    1.1494862,    3.531673 ,    1.5329071],
      dtype=float32)

In [14]:
l[855,1]

'siren'